In [4]:
rm(list = ls())

In [12]:
library(readxl)
library(dplyr)
library(tidyverse)

In [13]:
data_animals <- read_excel("summary_tables_linear_mixed_model/raw/Injured_F1.xlsx")
head(data_animals)

ID,Group,Father,Mother,Sex,Normalized_BW,Normalized_liver,Normalized_heart,Normalized_LV_area_systole,Normalized_LV_area_diastole,...,Camk1d,Triglycerides,Lipase,LDL,HDL,Total_Cholesterol,Free_Cholesterol,Calcium,Glucose,Free_fatty_acids
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
EQU 93.0,Pu-Fi,EQU 93,CX9576,M,0.43,NA,4.9285714285714297,0.61706550000000004,0.95233625,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
EQU 93.1,Pu-Fi,EQU 93,CX9576,M,0.481379310344828,NA,5.3724137931034504,0.68317941666666704,1.0450297500000001,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
EQU 93.2,Pu-Fi,EQU 93,CX9576,M,0.56240000000000001,NA,5.2880000000000003,0.460915153846154,0.863217846153846,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
EQU 93.3,Pu-Fi,EQU 93,CX9576,M,0.41769230769230797,NA,4.9538461538461496,0.62144130769230799,0.98138553846153898,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
EQU 93.4,Pu-Fi,EQU 93,CX9576,M,0.53391304347826096,NA,5.4173913043478299,0.70501599999999998,1.0909019130434801,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
EQU 93.5,Pu-Fi,EQU 93,CX9576,M,0.50076923076923097,NA,4.8692307692307697,0.55171953846153898,0.89632900000000004,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [16]:
# Convert all columns except for the first 5 to numeric
data_animals[] <- lapply(seq_along(data_animals), function(i) {
  if (i > 5 && is.character(data_animals[[i]])) {
    as.numeric(data_animals[[i]])
  } else {
    data_animals[[i]]
  }
})

Warning message in FUN(X[[i]], ...):
"NAs introduced by coercion"
Warning message in FUN(X[[i]], ...):
"NAs introduced by coercion"
Warning message in FUN(X[[i]], ...):
"NAs introduced by coercion"
Warning message in FUN(X[[i]], ...):
"NAs introduced by coercion"
Warning message in FUN(X[[i]], ...):
"NAs introduced by coercion"
Warning message in FUN(X[[i]], ...):
"NAs introduced by coercion"
Warning message in FUN(X[[i]], ...):
"NAs introduced by coercion"
Warning message in FUN(X[[i]], ...):
"NAs introduced by coercion"
Warning message in FUN(X[[i]], ...):
"NAs introduced by coercion"
Warning message in FUN(X[[i]], ...):
"NAs introduced by coercion"
Warning message in FUN(X[[i]], ...):
"NAs introduced by coercion"
Warning message in FUN(X[[i]], ...):
"NAs introduced by coercion"
Warning message in FUN(X[[i]], ...):
"NAs introduced by coercion"
Warning message in FUN(X[[i]], ...):
"NAs introduced by coercion"
Warning message in FUN(X[[i]], ...):
"NAs introduced by coercion"
Warning me

In [17]:
# Function to identify outliers using IQR method within groups
identify_outliers <- function(data_animals) {
  outlier_list <- list()
  
  # Loop through each numeric column
  for (column in colnames(data_animals)) {
    if (is.numeric(data_animals[[column]])) {
      
      # Calculate Q1, Q3, and IQR within each group
      outliers <- data_animals %>%
        group_by(Group) %>%
        mutate(
          Q1 = quantile(!!sym(column), 0.25, na.rm = TRUE),
          Q3 = quantile(!!sym(column), 0.75, na.rm = TRUE),
          IQR = Q3 - Q1,
          lower_bound = Q1 - 1.5 * IQR,
          upper_bound = Q3 + 1.5 * IQR
        ) %>%
        filter(!!sym(column) < lower_bound | !!sym(column) > upper_bound) %>%
        select(Group, column)
      
      # Store outliers in the list
      outlier_list[[column]] <- outliers
    }
  }
  
  return(outlier_list)
}
outliers <- identify_outliers(data_animals)

# Print the outliers
for (column in names(outliers)) {
  cat("\nOutliers in column:", column, "\n")
  print(outliers[[column]])
}



Outliers in column: Normalized_BW 
# A tibble: 0 x 2
# Groups:   Group [0]
# i 2 variables: Group <chr>, Normalized_BW <dbl>

Outliers in column: Normalized_liver 
# A tibble: 2 x 2
# Groups:   Group [1]
  Group Normalized_liver
  <chr>            <dbl>
1 Pi-Fi             65.8
2 Pi-Fi             63.0

Outliers in column: Normalized_heart 
# A tibble: 1 x 2
# Groups:   Group [1]
  Group Normalized_heart
  <chr>            <dbl>
1 Pi-Fi             9.36

Outliers in column: Normalized_LV_area_systole 
# A tibble: 2 x 2
# Groups:   Group [2]
  Group Normalized_LV_area_systole
  <chr>                      <dbl>
1 Pu-Fi                      0.928
2 Pi-Fi                      1.19 

Outliers in column: Normalized_LV_area_diastole 
# A tibble: 2 x 2
# Groups:   Group [1]
  Group Normalized_LV_area_diastole
  <chr>                       <dbl>
1 Pi-Fi                       1.84 
2 Pi-Fi                       0.658

Outliers in column: Normalized_LV_volume_systole 
# A tibble: 2 x 2
# Groups:

In [18]:
data_animals_uninjured <- read_excel("summary_tables_linear_mixed_model/raw/Uninjured_F1.xlsx")
head(data_animals_uninjured)

ID,Group,Father,Mother,Sex,Normalized_BW,Normalized_liver,Normalized_heart,Normalized_LV_area_systole,Normalized_LV_area_diastole,...,Camk1d,Triglycerides,Lipase,LDL,HDL,Total_Cholesterol,Free_Cholesterol,Calcium,Glucose,Free_fatty_acids
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
FFM 2.1,Pu-Fu,FFM2,CX88853,M,0.8728571,52.85714,5.714286,0.6713228,0.9271400,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
FFM 2.2,Pu-Fu,FFM2,CX88853,F,0.7921429,47.14286,5.714286,0.6245989,1.0321189,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
FFM 2.3,Pu-Fu,FFM2,CX88853,F,0.8115385,39.23077,5.384615,0.8538902,1.1825318,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
FFM 2.4,Pu-Fu,FFM2,CX88853,M,0.8828571,52.14286,5.714286,0.6428779,1.0398316,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
STOCK 1,Pu-Fu,Stock,Stock,M,0.7592857,NA,5.142857,0.6000033,0.9780934,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
STOCK 2,Pu-Fu,Stock,Stock,F,0.7185185,NA,5.037037,0.5374933,0.9650065,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [19]:
# Convert all columns except for the first 5 to numeric
data_animals_uninjured[] <- lapply(seq_along(data_animals_uninjured), function(i) {
  if (i > 5 && is.character(data_animals_uninjured[[i]])) {
    as.numeric(data_animals_uninjured[[i]])
  } else {
    data_animals_uninjured[[i]]
  }
})

In [21]:
# Function to identify outliers using IQR method within groups
identify_outliers <- function(data_animals_uninjured) {
  outlier_list <- list()
  
  # Loop through each numeric column
  for (column in colnames(data_animals_uninjured)) {
    if (is.numeric(data_animals_uninjured[[column]])) {
      
      # Calculate Q1, Q3, and IQR within each group
      outliers <- data_animals_uninjured %>%
        group_by(Group) %>%
        mutate(
          Q1 = quantile(!!sym(column), 0.25, na.rm = TRUE),
          Q3 = quantile(!!sym(column), 0.75, na.rm = TRUE),
          IQR = Q3 - Q1,
          lower_bound = Q1 - 1.5 * IQR,
          upper_bound = Q3 + 1.5 * IQR
        ) %>%
        filter(!!sym(column) < lower_bound | !!sym(column) > upper_bound) %>%
        select(Group, column)
      
      # Store outliers in the list
      outlier_list[[column]] <- outliers
    }
  }
  
  return(outlier_list)
}

outliers <- identify_outliers(data_animals_uninjured)

# Print the outliers
for (column in names(outliers)) {
  cat("\nOutliers in column:", column, "\n")
  print(outliers[[column]])
}


Outliers in column: Normalized_BW 
# A tibble: 0 x 2
# Groups:   Group [0]
# i 2 variables: Group <chr>, Normalized_BW <dbl>

Outliers in column: Normalized_liver 
# A tibble: 1 x 2
# Groups:   Group [1]
  Group Normalized_liver
  <chr>            <dbl>
1 Pi-Fu             44.2

Outliers in column: Normalized_heart 
# A tibble: 0 x 2
# Groups:   Group [0]
# i 2 variables: Group <chr>, Normalized_heart <dbl>

Outliers in column: Normalized_LV_area_systole 
# A tibble: 0 x 2
# Groups:   Group [0]
# i 2 variables: Group <chr>, Normalized_LV_area_systole <dbl>

Outliers in column: Normalized_LV_area_diastole 
# A tibble: 0 x 2
# Groups:   Group [0]
# i 2 variables: Group <chr>, Normalized_LV_area_diastole <dbl>

Outliers in column: Normalized_LV_volume_systole 
# A tibble: 0 x 2
# Groups:   Group [0]
# i 2 variables: Group <chr>, Normalized_LV_volume_systole <dbl>

Outliers in column: Normalized_LV_volume_diastole 
# A tibble: 0 x 2
# Groups:   Group [0]
# i 2 variables: Group <chr>, Norm